In [1]:
import torch
import esm
from torch import nn, optim
import random
import pandas as pd
from tqdm import tqdm

In [7]:
!head ../data/mariana_to_100.tsv

NLEIPMFD_00006 93 MCIFLRRAVLGCSCNYVLFMEYRTSDIQIAALLYAEGIELVNVDSSNPRRKEFVFKDEQQISELVKGFWQDTHKIAPRKYMGAFRELKNKLYS 0
NLEIPMFD_00010 86 MKYFVYLARCADGSLYTGSCTNIKAREIRHNKGEGAAFTYKRRPVKIVYFEEFKTLIEAMRREKQIKRWTRKKKENLVKYGHPTKF 0
NLEIPMFD_00012 77 MVFNVKRSTYVILTSFQMGKGDLKEKVITKFAYSTPSVTLALPTLFQSKALSAEQMRKLHATIFSVIDPPTQEAVES 0
NLEIPMFD_00013 71 MLMKGVQHKNPITPFGKIIMHCPDPTVLAISQAHLVPFLIQIFGQQQHLLFSPADVGSVRVKEDSHGSLND 0
NLEIPMFD_00015 42 MLPVSCVLEDWNYSPNYLGGVSIKNAGVPGNMLQGQSLNPNS 1
NLEIPMFD_00022 99 MEKLVNIVKRCLNHHKLGESAKASHVLFTAQQFLDKWFVGEKMMAKPVQLKNAVLWIGVRHPTIAQEFRGVSDKLLKELQTRFGPKLVQKIRTKHLTSI 0
NLEIPMFD_00025 96 MRLLLESIGFKVLEASNARHALTLINTEKPDITLTDHMMPGELTGEQLARHLHERGLKVVLTSGYPIEEESCFQFIAKPPRIGVLTAVLKKELGIE 0
NLEIPMFD_00029 95 MLDLTLRSKFLSPTTMIKEAVILKYGVVFTGKRHNVIFNSAQAMGLGFAGLRGGEQGFVTESGEFVNRRKAFEIALACGQIEEREKRKLFSEDLY 0
NLEIPMFD_00030 46 MSISSPAIRISGSGQIVQVLPMMKSRVITKFMGFSYLSYRTVKLVL 1
NLEIPMFD_00034 39 MEIPFIKVHPARLRAEEVLKDVIVQLKEMIEAGRIRNDL 1


In [2]:
# Example data: list of tuples (label, sequence)
data = [
    ("protein1", "MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPQYKGSGRTQY"),
    ("protein2", "GIEVVVNATLDKAGFQAGYIGFLKTFTLGVAGSGLLGGTY"),
    # Add more sequences here
]

# data = list()
# df_paths = ['../data/AMP_new/AMP_2024_08_09.tsv', '../data/mariana_to_100.tsv']
# for df_path,sep in zip(df_paths, ['\t', ' ']):
#     df = pd.read_csv(df_path, header=None, sep=sep)
#     for ind, row in df.iterrows():
#         data.append( (row[0], row[2]) )
#     # break

In [32]:
len(data)

1107014

In [21]:
len(data)

55209

In [4]:
# import os
# import sys

# current_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
# parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
# sibling_dir = os.path.join(parent_dir, 'src/embeddings')
# sys.path.insert(0, sibling_dir)
# sibling_dir = os.path.join(parent_dir, 'src')
# sys.path.insert(0, sibling_dir)
# import embedding_esm

/home/nikola_dev/miniconda3/envs/ssi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You are missing some of the libraries for ProteinVec
You are missing some of the libraries for ESM3


In [5]:
# esm_emb = embedding_esm.EsmEmbedding(esm_model_path="/home/nikola_dev/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D.pt")

In [14]:
from pathlib import Path
user_home = str(Path.home())
print(user_home)

esm_model_path=f"{user_home}/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D.pt"
model_data = torch.load(str(esm_model_path), map_location="cpu")

/home/nikola_dev


In [16]:
import torch
import esm
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
import random
import pandas as pd


# Check if a GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
# Load the pre-trained ESM-2 model and its alphabet
# model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
# cuz changed in EsmEmbedding
model = model.to(device)
batch_converter = alphabet.get_batch_converter()

# Convert the data to batch format
batch_labels, batch_strs, batch_tokens = batch_converter(data)

# Prepare a dataset and dataloader for batching
dataset = TensorDataset(batch_tokens)
batch_size = 4
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=alphabet.padding_idx)
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Masking function
def mask_tokens(tokens, mask_idx, pad_idx, mask_prob=0.15):
    labels = tokens.clone()
    masked_tokens = tokens.clone()

    # Create a mask based on the probability
    mask = (torch.rand(tokens.shape) < mask_prob) & (tokens != pad_idx)

    # Replace masked positions with the mask index
    masked_tokens[mask] = mask_idx

    return masked_tokens, labels

# Enable training mode
model.train()

# Fine-tuning loop
num_epochs = 5
mask_idx = alphabet.mask_idx# torch.tensor(alphabet.mask_idx).to(device)
pad_idx = alphabet.padding_idx #torch.tensor(alphabet.padding_idx).to(device)

for epoch in range(num_epochs):
    for batch in tqdm(dataloader):
        batch_tokens = batch[0]
        optimizer.zero_grad()

        # Mask tokens
        masked_tokens, labels = mask_tokens(batch_tokens, mask_idx, pad_idx)
        masked_tokens = masked_tokens.to(device)
        labels = labels.to(device)  # Move labels to GPU

        # Forward pass: get the output from the model
        # with torch.no_grad():
        output = model(masked_tokens, repr_layers=[33])
        logits = output["logits"]
        # Take the argmax of the logits to get the predicted amino acids
        # predictions = torch.argmax(logits, dim=-1)
        
        # print(logits.size(), labels.size())
        # print(logits.view(-1, logits.size(-1)).size(), labels.view(-1).size())
        # Compute loss for masked language modeling
        # argmax on 33 size vector (size of vocabulary) is performed inside CrossEntropyLoss function
        loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))  
        loss.backward()
        optimizer.step()
        
    print(f"Epoch: {epoch}, Loss: {loss.item()}")

# Save the fine-tuned model
# torch.save(model.state_dict(), "fine_tuned_esm2_masked_model.pth")
torch.save({
    "model": model.state_dict(),
    "args": model_data['args'],
    "cfg": model_data['cfg']
},
"esm2_ft.pt")

# Switch back to evaluation mode after fine-tuning
model.eval()

Using device: cuda


100%|██████████| 1/1 [00:00<00:00, 10.98it/s]


Epoch: 0, Loss: 0.7983145117759705


100%|██████████| 1/1 [00:00<00:00, 48.45it/s]


Epoch: 1, Loss: 0.6819889545440674


...

100%|██████████| 1/1 [00:00<00:00, 53.35it/s]


Epoch: 3, Loss: 0.6528562307357788


100%|██████████| 1/1 [00:00<00:00, 50.55it/s]


Epoch: 4, Loss: 0.8061037063598633


ESM2(
  (embed_tokens): Embedding(33, 320, padding_idx=1)
  (layers): ModuleList(
    (0-5): 6 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=320, out_features=320, bias=True)
        (v_proj): Linear(in_features=320, out_features=320, bias=True)
        (q_proj): Linear(in_features=320, out_features=320, bias=True)
        (out_proj): Linear(in_features=320, out_features=320, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=320, out_features=1280, bias=True)
      (fc2): Linear(in_features=1280, out_features=320, bias=True)
      (final_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=120, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((320,), eps=1e-05, elementwis

In [17]:
ft_model_path = "esm2_ft.pt"
model, alphabet = esm.pretrained.load_model_and_alphabet(ft_model_path)